In [15]:
import functools
import os
from typing import List

from eval import interpolator as interpolator_lib
from eval import util
from absl import logging
import apache_beam as beam
import mediapy as media
import natsort
import numpy as np
import tensorflow as tf
from tqdm.auto import tqdm

# Controls TF_CCP log level.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [16]:
pattern = r"\\shelter\Kyu\motility_interpolation\filmtest\*\original_frames"
directories = tf.io.gfile.glob(pattern)
directories

['\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy003\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy008\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy028\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy016\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy018\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy017\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy006\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy019\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy005\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy002\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy011\\original_frames',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest

In [14]:
n=2
times_to_interpolate=5
num_frames = (n - 1) * (2**(times_to_interpolate) - 1)
num_frames

31

In [5]:
def _output_frames(frames: List[np.ndarray], frames_dir: str):
  """Writes PNG-images to a directory.

  If frames_dir doesn't exist, it is created. If frames_dir contains existing
  PNG-files, they are removed before saving the new ones.

  Args:
    frames: List of images to save.
    frames_dir: The output directory to save the images.

  """
  # if tf.io.gfile.isdir(frames_dir):
  #   old_frames = tf.io.gfile.glob(f'{frames_dir}/frame_*.png')
  #   if old_frames:
  #     logging.info('Removing existing frames from %s.', frames_dir)
  #     for old_frame in old_frames:
  #       tf.io.gfile.remove(old_frame)
  # else:
  #   tf.io.gfile.makedirs(frames_dir)
  for idx, frame in tqdm(
      enumerate(frames), total=len(frames), ncols=100, colour='green'):
    util.write_image(f'{frames_dir}/im_{idx:02d}_{modelname}.png', frame)
  logging.info('Output frames saved in %s.', frames_dir)

class ProcessDirectory(beam.DoFn):
  """DoFn for running the interpolator on a single directory at the time."""

  def setup(self):
    self.interpolator = interpolator_lib.Interpolator(
        modelpath, align,
        [bh, bw])

    if outputvideo:
      ffmpeg_path = util.get_ffmpeg_path()
      media.set_ffmpeg(ffmpeg_path)

  def process(self, directory: str):
    input_frames_list = [
        natsort.natsorted(tf.io.gfile.glob(f'{directory}/*{ext}'))
        for ext in input_ext
    ]
    input_frames = functools.reduce(lambda x, y: x + y, input_frames_list)
    logging.info('Generating in-between frames for %s.', directory)
    frames = list(
        util.interpolate_recursively_from_files(
            input_frames, time2inter, self.interpolator))
    _output_frames(frames, f'{directory}')

    if outputvideo:
      media.write_video(f'{directory}/interpolated.mp4', frames, fps=fps)
      logging.info('Out;put video saved at %s/interpolated.mp4.', directory)

In [7]:
pattern = r"\\shelter\Kyu\motility_interpolation\filmtest_train_skip5\sequences\*\*"
directories = tf.io.gfile.glob(pattern)
directories = directories[:2]
print(directories)

modelpath=r"\\shelter\Kyu\motility_interpolation\pretrained_models\film_net\Style\saved_model"
# modelname=os.path.basename(modelpath)
modelname='ogfilm'
time2inter=5

align=64
bh=1
bw=1
outputvideo=False
fps=30
input_ext=['1.png','3.png']

pipeline = beam.Pipeline('DirectRunner')

['\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy001\\000', '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy001\\001']


In [8]:
(pipeline | 'Create directory names' >> beam.Create(directories)  # pylint: disable=expression-not-assigned
   | 'Process directories' >> beam.ParDo(ProcessDirectory()))

AttributeError: type object 'Series' has no attribute 'append'

In [ ]:
result = pipeline.run()
result.wait_until_finish()

In [17]:
directory=r'\\shelter\Kyu\motility_interpolation\filmtest\xy001\original_frames'
input_ext=['.png']
input_frames_list = [
        natsort.natsorted(tf.io.gfile.glob(f'{directory}/*{ext}'))
        for ext in input_ext
    ]
input_frames = functools.reduce(lambda x, y: x + y, input_frames_list)
input_frames

['\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t001xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t002xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t003xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t004xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t005xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t006xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t007xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t008xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t009xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\\original_frames\\t010xy001.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest\\xy001\

In [23]:
len(input_frames[::4])

31

In [24]:
(31 - 1) * (2**(2) - 1)

90